In [1]:
from datasets import load_dataset, load_metric
from fleurs import _FLEURS_LONG_TO_LANG
from config import STORAGE_DIR_DATA_FLEURS 
from models import load_whisper_x
import numpy as np

/home/keith/miniconda3/envs/keith/lib/python3.11/site-packages/pyannote/audio/core/io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
/home/keith/miniconda3/envs/keith/lib/python3.11/site-packages/pyannote/audio/pipelines/speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []
/home/keith/miniconda3/envs/keith/lib/python3.11/site-packages/pyannote/audio/pipelines/speaker_verification.py:45: UserWarning: Module 'speechbrain.pretrain

In [2]:
languages_long = "English, Mandarin Chinese".split(',')
languages_lang = [_FLEURS_LONG_TO_LANG[long.strip()] for long in languages_long]

dataset = load_dataset("google/fleurs", name=languages_lang[1], cache_dir=STORAGE_DIR_DATA_FLEURS, trust_remote_code=True, split="test[:10%]")

In [18]:
wer_metric = load_metric("wer")
cer_metric = load_metric("cer")

/home/keith/miniconda3/envs/keith/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for wer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/wer/wer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
/home/keith/miniconda3/envs/keith/lib/python3.11/site-packages/datasets/load.py:756: FutureWarning: The repository for cer contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.18.0/metrics/cer/cer.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to l

In [4]:
model = load_whisper_x(whisper_arch="medium", device="cpu")

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../../../.cache/torch/whisperx-vad-segmentation.bin`


No language specified, language will be first be detected for each audio file (increases inference time).
Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.1.1+cu121. Bad things might happen unless you revert torch to 1.x.


In [5]:
batch_size = 16
batch = []
results = []
for idx in range(0, 5):
    audio_array = dataset[idx]["audio"]["array"].astype(np.float32)
    result = model.transcribe(audio_array, batch_size=batch_size)

    transcription = result["segments"][0]['text']
    print(transcription)
    results.append(transcription)

Detected language: zh (0.98) in first 30s of audio...
这并不是告别这是一个篇章的结束也是新篇章的开始
Detected language: zh (1.00) in first 30s of audio...
盖甲等元素属于金属银和金等元素当然也是金属
Detected language: zh (0.99) in first 30s of audio...
橋下垂直淨空15米該項目於2011年8月完工但直到2017年3月才開始通車
Detected language: zh (0.99) in first 30s of audio...
适当使用博客可以使学生变得更善于分析和进行思辨,通过积极回忆网络材料,学生们可以在他人的文章的上下文语语境中找到自己的立场,并能够针对特定问题提出自己的观点。
Detected language: zh (0.99) in first 30s of audio...
科学家们可以得出结论暗物质对其他暗物质的影响方式与普通物质相同


In [26]:
references = dataset[0:5]["transcription"]
predictions = []

if dataset[0]["language"] == "Mandarin Chinese":
    for idx, reference in enumerate(results):
        # add spaces between characters
        def detect_chinese(text):
            has_chinese = any(
                "\u4E00" <= char <= "\u9FFF" or "\u3400" <= char <= "\u4DBF"
                for char in text
            )
            return has_chinese

        reconstruct = ""
        for idx, char in enumerate(reference):
            if detect_chinese(char):
                if idx != len(reference) - 1:
                    reconstruct += char + " "
                else:
                    reconstruct += char
            else:
                reconstruct += char

        predictions.append(reconstruct)

In [27]:
from pprint import pprint
pprint(dataset[0])

wer_result = wer_metric.compute(references=references, predictions=predictions)             
cer_result = cer_metric.compute(references=references, predictions=predictions)             
print(wer_result)
print(cer_result)

{'audio': {'array': array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       5.72800636e-05, 6.50286674e-05, 9.16719437e-05]),
           'path': 'test/10026684690566417990.wav',
           'sampling_rate': 16000},
 'gender': 0,
 'id': 1906,
 'lang_group_id': 6,
 'lang_id': 13,
 'language': 'Mandarin Chinese',
 'num_samples': 166080,
 'path': '/run/media/keith/a1cdd4e6-1c93-4b0a-8e04-b49b3dfb68d9/data/fleurs/downloads/extracted/6ac5a80ec9f5461d7471fc483e93eca5de8b6dd8de8ee1d2f25f3adc0961cdf9/10026684690566417990.wav',
 'raw_transcription': '“这并不是告别。这是一个篇章的结束，也是新篇章的开始。”',
 'transcription': '这 并 不 是 告 别 这 是 一 个 篇 章 的 结 束 也 是 新 篇 章 的 开 始'}
0.15
0.0891891891891892
